In [ ]:
# ## 프롬프트
# PROMPT_DIET_GENERATION = 
# """
# 너는 식단 추천 데이터 생성기다. 각 줄마다 JSON 객체 하나만 출력하고, 코드블록 없이 순수 텍스트로 NDJSON 형태를 반환하라.

# 필드 스키마:
# {
#   "image": "https://.../photo.jpg",                // VLM이 읽을 수 있는 이미지 URL (퍼블릭 또는 서명)
#   "diagnosis_name": "<string>",                    // 예: acne, atopic, none 등
#   "health_profile": "<string>",                    // 키/몸무게/활동/목표 요약
#   "calorie_plan": <int>,                           // 1식 목표 kcal 정수
#   "rules_text": "<string>",                        // 허용/제한/조건 텍스트
#   "diet_json": [
#     {
#       "menuName": "<string>",
#       "description": "<string>",
#       "calories": <int>,
#       "notes": "<string>"                          // 간단 코멘트, 필요 없으면 "" 또는 생략 가능
#     },
#     ... 3~4개
#   ]
# }

# 지시:
# - 각 객체는 위 필드를 모두 포함.
# - calories는 정수, menuName/description/notes는 한국어/영어 혼용 가능하나 간결하게.
# - recipeUrl 등 추가 필드는 넣지 말 것.
# - diagnosis_name, health_profile, rules_text, calorie_plan을 다양하게 섞어서 5개 이상의 샘플을 만들어라.
# - 출력은 JSON 객체 여러 개를 줄바꿈으로 이어붙인 NDJSON 한 덩어리로만 응답하라(코드블록 금지).
# """


위 프롬프트로 NDJSON을 만든다.  
필드는 diagnosis_name, health_profile, calorie_plan, rules_text, diet_json만 포함.

In [2]:
# !pip install pandas
from pathlib import Path
import json, itertools, random

In [3]:
# 1) 이미지 베이스 스캔 -> base NDJSON 생성
# mixture_data 폴더 구조를 스캔해 image 경로와 진단명을 매핑
root = Path("../mixture_data")

ACTIVITY = ["LOW", "MEDIUM", "HIGH"]
GOAL = ["LOSE", "MAINTAIN", "GAIN"]

def stub_profile():
    return {
        "height_cm": random.randint(160, 180),
        "weight_kg": random.randint(55, 75),
        "activity_level": random.choice(ACTIVITY),
        "goal_type": random.choice(GOAL),
    }

def rules_for(diag: str) -> str:
    if diag == "여드름":
        return "허용: 저당, 오메가3; 제한: 고당, 튀김; 조건: 수분 충분"
    if diag == "아토피":
        return "허용: 항염 음식; 제한: 가공식품/고당; 조건: 저염"
    return "허용: 채소, 살코기; 제한: 튀김/과당; 조건: 1식 칼로리 준수"

base_records = []
for img_path in root.glob("*/*"):
    if not img_path.is_file():
        continue
    diag = img_path.parent.name  # 한글 라벨 그대로
    prof = stub_profile()
    base_records.append({
        "image": str(img_path),
        "diagnosis_name": diag,
        **prof,
        "calorie_plan": random.randint(450, 650),
        "rules_text": rules_for(diag),
    })

out_base = root / "synthetic_base.ndjson"
out_base.parent.mkdir(exist_ok=True)
with out_base.open("w", encoding="utf-8") as f:
    for r in base_records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"wrote {len(base_records)} samples -> {out_base}")


wrote 600 samples -> ..\mixture_data\synthetic_base.ndjson


In [4]:
# 2) 텍스트 합성 프롬프트(복사해서 GPT 등에 사용)
PROMPT_DIET_GENERATION = """
너는 식단 추천 데이터 생성기다. 진단명은 반드시 {건선, 아토피, 여드름, 정상, 주사, 지루} 중 하나만 사용하라.
최소 200개 샘플을 생성하고, 진단 6종이 고르게 섞이도록 하라.
각 샘플은 JSON 객체 한 개로, NDJSON(줄바꿈으로 이어붙인 여러 JSON 객체) 한 덩어리로만 응답하라. 코드블록 금지.

필드 스키마:
{
  "diagnosis_name": "<건선|아토피|여드름|정상|주사|지루>",
  "health_profile": "<string>",          // 키/몸무게/활동/목표만 포함
  "calorie_plan": <int>,                 // 1식 kcal 정수, 다양한 범위 사용
  "rules_text": "<string>",              // 허용/제한/조건 텍스트
  "diet_json": [
    {"menuName": "<string>", "description": "<string>", "calories": <int>, "notes": "<string>"},
    ... 3~4개
  ]
}

지시:
- 진단명은 위 6개 중 하나만 사용하고, 각 진단이 여러 번 등장하도록 다양하게 섞는다.
- menuName은 언더스코어/숫자/라벨 접두어 없이 실제 음식명(예: 연어구이, 두부샐러드, 병아리콩스튜 등)만 사용
- menuName/description/notes/calories를 반복되지 않게 다양화한다(칼로리 범위도 350~700 정도로 다양).
- recipeUrl 등 추가 필드는 넣지 말 것.
- 출력은 NDJSON 한 덩어리(여러 줄의 JSON 객체)로만 응답하고, 앞뒤 여백/코드블록 없이 순수 텍스트로 반환한다.
"""
print(PROMPT_DIET_GENERATION)



너는 식단 추천 데이터 생성기다. 진단명은 반드시 {건선, 아토피, 여드름, 정상, 주사, 지루} 중 하나만 사용하라.
최소 200개 샘플을 생성하고, 진단 6종이 고르게 섞이도록 하라.
각 샘플은 JSON 객체 한 개로, NDJSON(줄바꿈으로 이어붙인 여러 JSON 객체) 한 덩어리로만 응답하라. 코드블록 금지.

필드 스키마:
{
  "diagnosis_name": "<건선|아토피|여드름|정상|주사|지루>",
  "health_profile": "<string>",          // 키/몸무게/활동/목표만 포함
  "calorie_plan": <int>,                 // 1식 kcal 정수, 다양한 범위 사용
  "rules_text": "<string>",              // 허용/제한/조건 텍스트
  "diet_json": [
    {"menuName": "<string>", "description": "<string>", "calories": <int>, "notes": "<string>"},
    ... 3~4개
  ]
}

지시:
- 진단명은 위 6개 중 하나만 사용하고, 각 진단이 여러 번 등장하도록 다양하게 섞는다.
- menuName은 언더스코어/숫자/라벨 접두어 없이 실제 음식명(예: 연어구이, 두부샐러드, 병아리콩스튜 등)만 사용
- menuName/description/notes/calories를 반복되지 않게 다양화한다(칼로리 범위도 350~700 정도로 다양).
- recipeUrl 등 추가 필드는 넣지 말 것.
- 출력은 NDJSON 한 덩어리(여러 줄의 JSON 객체)로만 응답하고, 앞뒤 여백/코드블록 없이 순수 텍스트로 반환한다.



In [5]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="./.env")  # 경로 명시
api_key = os.getenv("GMS_API_KEY")
print("has key:", bool(api_key), "len:", len(api_key) if api_key else 0)


has key: True len: 47


In [6]:
# 3) GMS api 호출을 통해서 합성데이터 생성
import os, asyncio, json
from pathlib import Path
from dotenv import load_dotenv
from openai import AsyncOpenAI

async def main():
    load_dotenv(dotenv_path="./.env") 
    api_key = os.getenv("GMS_API_KEY")
    assert api_key, "GMS_API_KEY 없음"

    client = AsyncOpenAI(
        base_url="https://gms.ssafy.io/gmsapi/api.openai.com/v1",
        api_key=api_key,
    )

    stream = await client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "너는 식단 추천 데이터 생성기다. NDJSON만 응답."},
            {"role": "user", "content": PROMPT_DIET_GENERATION},
        ],
        stream=True,
    )

    res_text = ""
    async for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            res_text += delta

    out = Path("../mixture_data/synthetic_text.ndjson")
    out.parent.mkdir(exist_ok=True)
    out.write_text(res_text, encoding="utf-8")
    print("saved ->", out)

await main()

saved -> ..\mixture_data\synthetic_text.ndjson


In [7]:
# 4) 텍스트 합성 NDJSON 불러와서 이미지 베이스와 병합
from pathlib import Path
import json, itertools

# 경로 설정
base_path = Path("../mixture_data/synthetic_base.ndjson")
text_path = Path("../mixture_data/synthetic_text.ndjson")
out_final = Path("../mixture_data/synthetic_final.ndjson")

# 로드
base_records = [json.loads(l) for l in base_path.open(encoding="utf-8") if l.strip()]
text_rows   = [json.loads(l) for l in text_path.open(encoding="utf-8") if l.strip()]

# 병합
final_records = []
text_cycle = itertools.cycle(text_rows)
for b in base_records:
    t = next(text_cycle)
    final_records.append({
        "image": b["image"],
        "diagnosis_name": b["diagnosis_name"],
        "height_cm": b["height_cm"],
        "weight_kg": b["weight_kg"],
        "activity_level": b["activity_level"],
        "goal_type": b["goal_type"],
        "calorie_plan": t.get("calorie_plan", b["calorie_plan"]),
        "rules_text": t.get("rules_text", b["rules_text"]),
        "diet_json": t["diet_json"],
    })

# 저장
out_final.parent.mkdir(exist_ok=True)
out_final.write_text("\n".join(json.dumps(r, ensure_ascii=False) for r in final_records), encoding="utf-8")
print(f"merged {len(final_records)} samples -> {out_final}")



merged 600 samples -> ..\mixture_data\synthetic_final.ndjson


In [8]:
# 5) 데이터셋 검증
import json, re
from collections import Counter

rows = [json.loads(l) for l in open("../mixture_data/synthetic_text.ndjson", encoding="utf-8") if l.strip()]
labels = Counter(r["diagnosis_name"] for r in rows)
print(labels)

bad_menu = []
for i,r in enumerate(rows):
    for item in r["diet_json"]:
        if re.search(r"[_0-9]", item["menuName"]):
            bad_menu.append((i, item["menuName"]))
print("bad menu count:", len(bad_menu))

cal_ok = all(isinstance(item["calories"], int) for r in rows for item in r["diet_json"])
print("calories int:", cal_ok)


Counter({'건선': 15, '아토피': 15, '여드름': 15, '정상': 15, '주사': 15, '지루': 15})
bad menu count: 0
calories int: True
